In [2]:
import requests
import pandas as pd
import json

In [3]:
cats = requests.get('https://api.mercadolibre.com/sites/MLA/categories')
cats.json()

[{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
 {'id': 'MLA1512', 'name': 'Agro'},
 {'id': 'MLA1403', 'name': 'Alimentos y Bebidas'},
 {'id': 'MLA1071', 'name': 'Animales y Mascotas'},
 {'id': 'MLA1367', 'name': 'Antigüedades y Colecciones'},
 {'id': 'MLA1368', 'name': 'Arte, Librería y Mercería'},
 {'id': 'MLA1743', 'name': 'Autos, Motos y Otros'},
 {'id': 'MLA1384', 'name': 'Bebés'},
 {'id': 'MLA1246', 'name': 'Belleza y Cuidado Personal'},
 {'id': 'MLA1039', 'name': 'Cámaras y Accesorios'},
 {'id': 'MLA1051', 'name': 'Celulares y Teléfonos'},
 {'id': 'MLA1648', 'name': 'Computación'},
 {'id': 'MLA1144', 'name': 'Consolas y Videojuegos'},
 {'id': 'MLA1500', 'name': 'Construcción'},
 {'id': 'MLA1276', 'name': 'Deportes y Fitness'},
 {'id': 'MLA5726', 'name': 'Electrodomésticos y Aires Ac.'},
 {'id': 'MLA1000', 'name': 'Electrónica, Audio y Video'},
 {'id': 'MLA2547', 'name': 'Entradas para Eventos'},
 {'id': 'MLA407134', 'name': 'Herramientas'},
 {'id': 'MLA1574', 'name': 'H

In [115]:
#Data Collection


    

cat_id = 'MLA3947'
offset = 50
url = f'https://api.mercadolibre.com/sites/MLA/search?category={cat_id}&offset={offset}'
request = requests.get(url)
items = request.json()
filtered_items = []
for item in items['results']:
    attributes = item.get('attributes', [])
    filtered_item = {
        'MLA_id': item['id'],
        'title': item['title'],
        'condition': item['condition'],
        'permalink': item['permalink'],
        'category_id': item['category_id'],
        'domain_id': item['domain_id'],
        'tags': item['tags'],
        'thumbnail': item['thumbnail'],
        'price': item['price'],
        'brand': next((attr['value_name'] for attr in attributes if attr['id'] == 'BRAND'), None),
        'model': next((attr['value_name'] for attr in attributes if attr['id'] == 'MODEL'), None)
    }
    filtered_items.append(filtered_item)

df_ini = pd.DataFrame(filtered_items)
df_ini['image_url'] = df_ini.apply(lambda row: get_image(row['MLA_id']), axis=1)


# Concatenate the two DataFrames
result_df = pd.concat([df_ini, df_picture_urls], axis=1)

result_df

,MLA_id,title,condition,permalink,category_id,domain_id,tags,thumbnail,price,brand,model,image_url,0
0,MLA721391285,Prendedor Pin Bandera Escarapela Argentina Str...,new,https://articulo.mercadolibre.com.ar/MLA-72139...,MLA3947,MLA-BROOCHES,"[brand_verified, loyalty_discount_eligible, po...",http://http2.mlstatic.com/D_718831-MLA31083675...,3310.92,pez globo,Strass,[https://http2.mlstatic.com/D_718831-MLA310836...,"{'MLA_id': 'MLA721391285', 'picture_1': 'https..."
1,MLA628052942,Escarapela La Duarte Pin Prendedor Cristal Arg...,new,https://articulo.mercadolibre.com.ar/MLA-62805...,MLA3947,MLA-BROOCHES,"[brand_verified, loyalty_discount_eligible, po...",http://http2.mlstatic.com/D_653382-MLA31124193...,5900.00,La Duarte,Escarapela N&P strass,[https://http2.mlstatic.com/D_996561-MLA694287...,"{'MLA_id': 'MLA628052942', 'picture_1': 'https..."
2,MLA721394101,Prendedor Pin Bandera Escarapela Argentina Str...,new,https://articulo.mercadolibre.com.ar/MLA-72139...,MLA3947,MLA-BROOCHES,"[brand_verified, poor_quality_thumbnail, stand...",http://http2.mlstatic.com/D_614097-MLA32239042...,2091.00,pez globo,Strass,[https://http2.mlstatic.com/D_614097-MLA322390...,"{'MLA_id': 'MLA721394101', 'picture_1': 'https..."
3,MLA1145523112,Broche Alfiler Para Saco 2,new,https://articulo.mercadolibre.com.ar/MLA-11455...,MLA3947,MLA-BROOCHES,"[incomplete_technical_specs, poor_quality_pict...",http://http2.mlstatic.com/D_961178-MLA50593872...,3500.00,None,None,[https://http2.mlstatic.com/D_961178-MLA505938...,"{'MLA_id': 'MLA1145523112', 'picture_1': 'http..."
4,MLA1203681151,Lazo Rosa En Favor De La Lucha Contra El Cance...,new,https://articulo.mercadolibre.com.ar/MLA-12036...,MLA3947,MLA-BROOCHES,"[poor_quality_thumbnail, poor_quality_picture,...",http://http2.mlstatic.com/D_805707-MLA52045944...,1800.00,celeste y blanca,lazo,[https://http2.mlstatic.com/D_805707-MLA520459...,"{'MLA_id': 'MLA1203681151', 'picture_1': 'http..."
5,MLA1422233954,Escarapela Argentina Pin Prendedor,new,https://articulo.mercadolibre.com.ar/MLA-14222...,MLA3947,MLA-BROOCHES,"[3x_campaign, poor_quality_thumbnail, immediat...",http://http2.mlstatic.com/D_809597-MLA70286247...,5900.00,Genérica,Escarapela Argentina,[https://http2.mlstatic.com/D_809597-MLA702862...,"{'MLA_id': 'MLA1422233954', 'picture_1': 'http..."
6,MLA1437807234,Prendedor Escarapela Argentina Esmaltada - Acero,new,https://articulo.mercadolibre.com.ar/MLA-14378...,MLA3947,MLA-BROOCHES,"[poor_quality_thumbnail, immediate_payment, ca...",http://http2.mlstatic.com/D_710582-MLA70107109...,1999.00,ESCARAPELA,ESCARAPELA ARGENTINA,[https://http2.mlstatic.com/D_710582-MLA701071...,"{'MLA_id': 'MLA1437807234', 'picture_1': 'http..."
7,MLA1138247608,500 Escarapelas Lazo Con Alfiler Para El 25 De...,new,https://articulo.mercadolibre.com.ar/MLA-11382...,MLA3947,MLA-BROOCHES,"[poor_quality_thumbnail, immediate_payment, ca...",http://http2.mlstatic.com/D_852646-MLA50025368...,6900.00,celeste y blanca,lazo,[https://http2.mlstatic.com/D_852646-MLA500253...,"{'MLA_id': 'MLA1138247608', 'picture_1': 'http..."
8,MLA1141040007,Escarapela Prendedor Cristal Argentina Con Sol,new,https://articulo.mercadolibre.com.ar/MLA-11410...,MLA3947,MLA-BROOCHES,"[poor_quality_thumbnail, immediate_payment, ca...",http://http2.mlstatic.com/D_662528-MLA50212573...,6999.99,lmcreacionesjoyas,Escarapela strass con sol prendedor,[https://http2.mlstatic.com/D_662528-MLA502125...,"{'MLA_id': 'MLA1141040007', 'picture_1': 'http..."
9,MLA1141059147,Bandera Argentina Grande Con Sol Bijouterie P...,new,https://articulo.mercadolibre.com.ar/MLA-11410...,MLA3947,MLA-BROOCHES,"[poor_quality_thumbnail, immediate_payment, ca...",http://http2.mlstatic.com/D_786625-MLA50212379...,7699.99,lmcreacionesjoyas,Bandera argentina grande strass,[https://http2.mlstatic.com/D_764203-MLA502123...,"{'MLA_id': 'MLA1141059147', 'picture_1': 'http..."


In [114]:
def get_image(id_sku):
    url = f'https://api.mercadolibre.com/items/{id_sku}'
    request = requests.get(url)
    item = request.json()
    
    picture_urls = []
    pictures = item.get('pictures', [])[:1]  # Limitar a solo 1 foto, opcional más a futuro
    for i, pic in enumerate(pictures, start=1):
        #Hace append del valor secure_url al array
        picture_urls.append(pic.get('secure_url', None))
    
    return picture_urls

# Replace 'your_id_sku_here' with the actual SKU ID
sku_id = 'MLA721391285'
item_data = get_image(sku_id)
item_data


['https://http2.mlstatic.com/D_718831-MLA31083675014_062019-O.jpg']

In [105]:
#Unir ambas funciones para generar el df inicial con la info necesaria


def get_image(id_sku):
    url = f'https://api.mercadolibre.com/items/{id_sku}'
    request = requests.get(url)
    item = request.json()
    
    picture_urls = []
    pictures = item.get('pictures', [])[:1]  # Limitar a solo 1 foto, opcional más a futuro
    for i, pic in enumerate(pictures, start=1):
        #Hace append del valor secure_url al array
        picture_urls.append(pic.get('secure_url', None))
    
    return {'MLA_id': id_sku, 'picture_1': picture_urls[0] if len(picture_urls) > 0 else None}



def initial_data(cat_id,offset):
    url = f'https://api.mercadolibre.com/sites/MLA/search?category={cat_id}&offset={offset}'
    request = requests.get(url)
    items = request.json()
    filtered_items = []
    for item in items['results']:
        attributes = item.get('attributes', [])
        filtered_item = {
            'MLA_id': item['id'],
            'title': item['title'],
            'condition': item['condition'],
            'permalink': item['permalink'],
            'category_id': item['category_id'],
            'domain_id': item['domain_id'],
            'tags': item['tags'],
            'thumbnail': item['thumbnail'],
            'price': item['price'],
            'brand': next((attr['value_name'] for attr in attributes if attr['id'] == 'BRAND'), None),
            'model': next((attr['value_name'] for attr in attributes if attr['id'] == 'MODEL'), None)
        }
        filtered_items.append(filtered_item)

    df_ini = pd.DataFrame(filtered_items)
    df_picture_urls = df_ini.apply(lambda row: get_image(row['MLA_id']), axis=1)
    df_picture_1 = df_picture_urls[['picture_1']]

    # Concatenate the two DataFrames
    result_df = pd.concat([df_ini, df_picture_1], axis=1)
    return result_df
   





In [103]:

cat_id = 'MLA3947'
offset = 50
a = initial_data(cat_id,offset)

In [104]:
a.head()

,MLA_id,title,condition,permalink,category_id,domain_id,tags,thumbnail,price,brand,model,0
0,MLA721391285,Prendedor Pin Bandera Escarapela Argentina Str...,new,https://articulo.mercadolibre.com.ar/MLA-72139...,MLA3947,MLA-BROOCHES,"[brand_verified, loyalty_discount_eligible, po...",http://http2.mlstatic.com/D_718831-MLA31083675...,3310.92,pez globo,Strass,"{'MLA_id': 'MLA721391285', 'picture_1': 'https..."
1,MLA628052942,Escarapela La Duarte Pin Prendedor Cristal Arg...,new,https://articulo.mercadolibre.com.ar/MLA-62805...,MLA3947,MLA-BROOCHES,"[brand_verified, loyalty_discount_eligible, po...",http://http2.mlstatic.com/D_653382-MLA31124193...,5900.00,La Duarte,Escarapela N&P strass,"{'MLA_id': 'MLA628052942', 'picture_1': 'https..."
2,MLA721394101,Prendedor Pin Bandera Escarapela Argentina Str...,new,https://articulo.mercadolibre.com.ar/MLA-72139...,MLA3947,MLA-BROOCHES,"[brand_verified, poor_quality_thumbnail, stand...",http://http2.mlstatic.com/D_614097-MLA32239042...,2091.00,pez globo,Strass,"{'MLA_id': 'MLA721394101', 'picture_1': 'https..."
3,MLA1145523112,Broche Alfiler Para Saco 2,new,https://articulo.mercadolibre.com.ar/MLA-11455...,MLA3947,MLA-BROOCHES,"[incomplete_technical_specs, poor_quality_pict...",http://http2.mlstatic.com/D_961178-MLA50593872...,3500.00,None,None,"{'MLA_id': 'MLA1145523112', 'picture_1': 'http..."
4,MLA1203681151,Lazo Rosa En Favor De La Lucha Contra El Cance...,new,https://articulo.mercadolibre.com.ar/MLA-12036...,MLA3947,MLA-BROOCHES,"[poor_quality_thumbnail, poor_quality_picture,...",http://http2.mlstatic.com/D_805707-MLA52045944...,1800.00,celeste y blanca,lazo,"{'MLA_id': 'MLA1203681151', 'picture_1': 'http..."
